In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import re
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [24]:
import os
from tqdm import tqdm
import pandas as pd
import argparse
from transformers import PreTrainedTokenizerFast
import argparse
import datetime
import pandas as pd
from huggingface_hub import hf_hub_download

repo = "ai4bharat/sangraha"
sb_folder = "verified/tel"
local_dir = "ai4bharat"

In [25]:
def get_dataset(repo, sb_folder, local_dir, range_):
    telugu_char_pattern = re.compile(r'[^\u0C00-\u0C7F\u0000-\u007F\s<>\n0-9.,!?]')
    filenames = [f"data-{i}.parquet" for i in range(*range_)]
    df_dataset = pd.DataFrame()
    for filename in filenames:
        file_path = hf_hub_download(
            repo_id=repo,
            repo_type="dataset",
            subfolder=sb_folder,
            filename=filename,
            local_dir=local_dir
        )
        df = pd.read_parquet(file_path)
        df = df.rename(columns={"text": "Input"})
        df.drop(columns=['type'], inplace=True)
        df.drop(columns=['doc_id'], inplace=True)
        df["Input"] = "<bos> " + df["Input"] + " <eos>"
        df["Input"] = df["Input"].str.replace("\n", "<newline>")
        df["Input"] = df["Input"].apply(lambda x: telugu_char_pattern.sub("", x))
        df_dataset = pd.concat([df_dataset, df], ignore_index=True)
    
    return df_dataset

In [26]:
df = get_dataset(repo, sb_folder, local_dir, (13, 14))

In [27]:
df

,Input
0,<bos> వెండితెరపై లవర్ బాయ్స్ చాలామంది ఉన్నారు....
1,<bos> ఎల్ఐసీ పాలసీని మధ్యలోనే ఆపేశారా?<newline...
2,<bos> సరికాని ఆహారం మరియు ఉత్పత్తుల నాణ్యమైన న...
3,<bos> మహేష్ సినిమాలో విజయశాంతి పాత్ర ఇదే!<newl...
4,<bos> ఢిల్లీ : భారతీయ రైల్వే మరో ఘనతను సాధించి...
...,...
174757,"<bos> ఎ. వెంక టేశ్వరరావు (విజయవాడ), ఎస్. సత్యన..."
174758,<bos> భారత ప్రధాని హోదాలో అగ్రరాజ్యం అమెరికా ప...
174759,<bos> నవతరం దర్శకుల్లో తనదైన అభిరుచిని చాటుకుం...
174760,<bos> తన భార్య అనుష్క డెలివరీ నేపథ్యంలో విరాట్...


In [22]:
unique_chars = set()
for i in range(len(df)):
    unique_chars.update(set(df.iloc[i]['Input']))

unique_chars = sorted(list(unique_chars))
print(unique_chars)

['\t', '\x1f', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', 'ఁ', 'ం', 'ః', 'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ఌ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ', 'క', 'ఖ', 'గ', 'ఘ', 'ఙ', 'చ', 'ఛ', 'జ', 'ఝ', 'ఞ', 'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల', 'ళ', 'ఴ', 'వ', 'శ', 'ష', 'స', 'హ', 'ఽ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ౄ', 'ె', 'ే', 'ై', 'ొ', 'ో', 'ౌ', '్', 'ౕ', 'ౖ', 'ౘ', 'ౙ', 'ౠ', 'ౡ', 'ౢ', '౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯', '౹', '౼', '౽']


In [28]:
import sys

def get_size(df):
    return sys.getsizeof(df) / 1024 / 1024

get_size(df)

713.6504907608032

In [29]:
rows = df.shape[0]

rows_150 = 150/get_size(df) * rows
rows_300 = 300/get_size(df) * rows
rows_500 = 500/get_size(df) * rows
print(rows_150, rows_300, rows_500)

df150 = df.sample(int(rows_150))
df300 = df.sample(int(rows_300))
df500 = df.sample(int(rows_500))
print(get_size(df150), get_size(df300), get_size(df500))

36732.6868535516 73465.3737071032 122442.28951183865
150.839937210083 301.4163007736206 500.55063247680664


In [30]:
text_150 = df150['Input'].tolist()
text_300 = df300['Input'].tolist()
text_500 = df500['Input'].tolist()

In [31]:
from tokenizers import SentencePieceBPETokenizer, ByteLevelBPETokenizer, Tokenizer, CharBPETokenizer
from transformers import PreTrainedTokenizerFast
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
import sentencepiece as spm


def train_tokenizer(data_list, tokenizer_name, vocab_size = 32768, model_name = "tokenizer"):
    if not os.path.exists(model_name):
        os.makedirs(model_name)

    bos_tok = "<bos>"
    eos_tok = "<eos>"

    special_char = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

    if (tokenizer_name == "SentencePieceBPETokenizer"):
        tokenizer = SentencePieceBPETokenizer()
        
    if (tokenizer_name == "ByteLevelBPETokenizer"):
        tokenizer = ByteLevelBPETokenizer()
        
    if (tokenizer_name == "CharBPETokenizer"):
        tokenizer = CharBPETokenizer()
        
    if (tokenizer_name == "WordPiece"):
        tokenizer = Tokenizer(WordPiece(unk_token="<unk>"))
        trainer = WordPieceTrainer(
            vocab_size=vocab_size,
            special_tokens=[bos_tok, eos_tok, "<unk>", "<pad>", "<newline>"] + special_char,
        )
        tokenizer.train_from_iterator(data_list, trainer=trainer)
        transformer_tokenizer = PreTrainedTokenizerFast(
            tokenizer_object=tokenizer,
            bos_token = bos_tok,
            eos_token = eos_tok,
            unk_token = "<unk>",
            pad_token = "<pad>",
            mask_token = "<mask>",
            padding_side = "left",
            truncation_side = "right",
            clean_up_tokenization_spaces = False,
        )

        transformer_tokenizer.save_pretrained(model_name)
        print(f"Tokenizer saved to {model_name}")
        return
        
    if (tokenizer_name == "SentencePiece"):
        spm.SentencePieceTrainer.train(
            input=data_list,
            model_prefix=model_name,
            vocab_size=vocab_size,
            bos_id=0,
            eos_id=1,
            pad_id=2,
            unk_id=3,
            character_coverage=1.0,
            model_type="bpe",
            user_defined_symbols=special_char,
        )
        return
    
    tokenizer.train_from_iterator(
        data_list,
        vocab_size = vocab_size,
        min_frequency = 5,
        special_tokens = ["<pad>", "<unk>", bos_tok, eos_tok, "<newline>"] + special_char,
        show_progress = True,
    )

    transformer_tokenizer = PreTrainedTokenizerFast(
        tokenizer_object=tokenizer,
        bos_token = bos_tok,
        eos_token = eos_tok,
        unk_token = "<unk>",
        pad_token = "<pad>",
        mask_token = "<mask>",
        padding_side = "left",
        truncation_side = "right",
        clean_up_tokenization_spaces = False,
    )

    transformer_tokenizer.save_pretrained(model_name)
    print(f"Tokenizer saved to {model_name}")

In [32]:
from transformers import AutoTokenizer

def get_fertility_score(texts,model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    total_tokens = 0
    total_words = 0
    
    for text in texts:
        tokens = tokenizer.tokenize(text)
        total_tokens += len(tokens)
        
        words = text.split()
        total_words += len(words)
        
    fertility_score = total_tokens / total_words
    
    print(f"Fertility Score of {model_name}: {fertility_score:.2f}")

    return fertility_score

In [33]:
import shutil
import csv

# tokenizer_names = ["SentencePieceBPETokenizer", "ByteLevelBPETokenizer", "CharBPETokenizer", "WordPiece", "SentencePiece"]
tokenizer_names = ["SentencePieceBPETokenizer"]
df = pd.DataFrame(columns=["Tokenizer", "Text", "Fertility Score"])
fertility_scores = dict()
for tokenizer_name in tokenizer_names:
    for text, text_name, df_subset in zip([text_150, text_300, text_500], ["150", "300", "500"], [df150, df300, df500]):
        model_name = f"{tokenizer_name}_{text_name}"
        train_tokenizer(text, tokenizer_name, model_name=model_name)
        shutil.make_archive(model_name, 'zip', model_name)
        fertility_scores[model_name] = get_fertility_score(text, model_name)
        new_row = pd.DataFrame({"Tokenizer": [tokenizer_name], "Text": [text_name], "Fertility Score": [fertility_scores[model_name]]})
        df = pd.concat([df, new_row], ignore_index=True)
        df.to_csv(f"fertility_scores_{model_name}.csv", index=False)
        print()




Tokenizer saved to SentencePieceBPETokenizer_150
Fertility Score of SentencePieceBPETokenizer_150: 1.68



/var/folders/rn/v0v0mhyx1lq27mkj7vl7v5c80000gn/T/ipykernel_85815/1258564952.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)





Tokenizer saved to SentencePieceBPETokenizer_300
Fertility Score of SentencePieceBPETokenizer_300: 1.68




Tokenizer saved to SentencePieceBPETokenizer_500
Fertility Score of SentencePieceBPETokenizer_500: 1.68

